In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/captcha/large/train"
!pwd

 Bicycle   Bus	 Chimney     Hydrant	  Mountain   Palm    'Traffic Light'
 Bridge    Car	 Crosswalk   Motorcycle   Other      Stairs
/content


In [ ]:
import os
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

import matplotlib.pyplot as plt
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import Model 

In [ ]:
large_labels = ['Bicycle', 'Bridge', 'Bus', 'Car', 'Chimney', 'Crosswalk', 'Hydrant', 'Motorcycle', 'Mountain', 'Other', 'Palm', 'Stairs', 'Traffic Light']
image_size = 150
batch_size = 20

np_data = 'numpy_data/'

if (not os.path.exists(np_data)):
      os.mkdir(np_data)

large_train_path = "/content/drive/MyDrive/captcha/large/train"
large_test_path = "/content/drive/MyDrive/captcha/large/val"

In [ ]:
def save_data(data_dir, file_prefix, labels):
    data = [] 
    
    for label in labels:
      
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                #img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                img_path = os.path.join(path, img)
                img = keras.preprocessing.image.load_img(img_path, target_size=(image_size, image_size))
                img_arr = keras.preprocessing.image.img_to_array(img)
                img_arr = img_arr/255.0
                #img_arr = cv2.imread(os.path.join(path,img),1)
                #resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                #data.append([resized_arr, class_num])
                data.append([img_arr, class_num])
            except Exception as e:
                print(e)
        np.save(np_data+file_prefix+label, data)
    return np.array(data)

In [ ]:
large_train_data = save_data(large_train_path,'Large_Train_',large_labels)
large_test_data = save_data(large_test_path,'Large_Test_',large_labels)

/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(directory=large_train_path, target_size=(image_size, image_size))

Found 16439 images belonging to 13 classes.


In [ ]:
validation_datagen = keras.preprocessing.image.ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(directory=large_test_path, target_size=(image_size, image_size))

Found 4117 images belonging to 13 classes.


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [ ]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
inc_history = model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
100/100 [==============================] - 299s 3s/step - loss: 31.3018 - acc: 0.8390 - val_loss: 42.5239 - val_acc: 0.9231
Epoch 2/10
100/100 [==============================] - 298s 3s/step - loss: 28.5569 - acc: 0.8482 - val_loss: 10.3235 - val_acc: 0.9198
Epoch 3/10
100/100 [==============================] - 299s 3s/step - loss: 24.6402 - acc: 0.8467 - val_loss: 10.9857 - val_acc: 0.6228
Epoch 4/10
100/100 [==============================] - 297s 3s/step - loss: 20.9320 - acc: 0.8453 - val_loss: 40.3442 - val_acc: 0.9231
Epoch 5/10
100/100 [==============================] - 294s 3s/step - loss: 16.0198 - acc: 0.8512 - val_loss: 20.3948 - val_acc: 0.9231
Epoch 6/10
100/100 [==============================] - 288s 3s/step - loss: 12.5177 - acc: 0.8472 - val_loss: 15.3017 - val_acc: 0.9231
Epoch 7/10
100/100 [==============================] - 296s 3s/step - loss: 9.8832 - acc: 0.8480 - val_loss: 8.1782 - val_acc: 0.9231
Epoch 8/10
100/100 [==============================] - 289

In [ ]:
model.save('inception_CaptchaModel_v1.h5')